# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import sys
sys.path.append('../')
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kumi,36.1136,128.3360,18.75,52,100,1.68,KR,1712805512
1,1,puerto ayora,-0.7393,-90.3518,24.75,98,81,1.52,EC,1712805513
2,2,okha,53.5739,142.9478,5.70,72,10,2.68,RU,1712805513
3,3,port-aux-francais,-49.3500,70.2167,6.87,94,100,5.75,TF,1712805513
4,4,khandyga,62.6667,135.6000,-1.03,84,86,2.32,RU,1712805513
...,...,...,...,...,...,...,...,...,...,...
583,583,hamza,40.4264,71.4956,17.97,40,1,0.62,UZ,1712805588
584,584,yichun,47.7000,128.9000,16.17,32,87,5.04,CN,1712805588
585,585,cholet,47.1667,-1.0000,11.84,73,100,3.38,FR,1712805588
586,586,thanh hoa,19.8000,105.7667,26.72,75,100,1.35,VN,1712805588


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:

map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700, #optional
    frame_height = 500 #optional
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
filtered_cities = city_data_df[city_data_df['Cloudiness'] == 0
                               & (city_data_df['Max Temp'] <27) & (city_data_df['Max Temp'] >21) & (city_data_df['Wind Speed'] <4)]
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,labrador city,52.9463,-66.9114,-1.01,51,0,3.58,CA,1712805514
17,17,an nuhud,12.7000,28.4333,22.83,11,0,6.08,SD,1712805514
26,26,kodiak,57.7900,-152.4072,2.73,48,0,4.12,US,1712805515
34,34,chibougamau,49.9168,-74.3659,3.79,65,0,2.06,CA,1712805516
68,68,tomatlan,19.9333,-105.2500,18.34,89,0,0.66,MX,1712805287
...,...,...,...,...,...,...,...,...,...,...
540,540,raton,36.9034,-104.4391,5.19,36,0,2.57,US,1712805581
543,543,cuatro cienegas de carranza,26.9833,-102.0833,20.91,15,0,2.90,MX,1712805582
552,552,la cruz,23.9167,-106.9000,17.70,71,0,0.58,MX,1712805583
562,562,crane,31.3974,-102.3501,17.39,27,0,4.25,US,1712805585


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,labrador city,CA,52.9463,-66.9114,51,
17,an nuhud,SD,12.7000,28.4333,11,
26,kodiak,US,57.7900,-152.4072,48,
34,chibougamau,CA,49.9168,-74.3659,65,
68,tomatlan,MX,19.9333,-105.2500,89,
...,...,...,...,...,...,...
540,raton,US,36.9034,-104.4391,36,
543,cuatro cienegas de carranza,MX,26.9833,-102.0833,15,
552,la cruz,MX,23.9167,-106.9000,71,
562,crane,US,31.3974,-102.3501,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
#Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    radius = 10000
    params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel",
    "limit": 20
    }

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

labrador city - nearest hotel: Carol Inn
an nuhud - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
chibougamau - nearest hotel: No hotel found
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
nerac - nearest hotel: Les Terrasses du Petit Nérac
hadibu - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
conde - nearest hotel: Paladino
saint-pierre - nearest hotel: Tropic Hotel
mountain home - nearest hotel: Best Western Foothills Inn
port elizabeth - nearest hotel: Waterford Hotel
broken hill - nearest hotel: Ibis Styles
valentin gomez farias - nearest hotel: No hotel found
walvis bay - nearest hotel: Atlantic
lapseki - nearest hotel: Orcin Apart Hotel
cabo san lucas - nearest hotel: Comfort Rooms
rio gallegos - nearest hotel: Amérian Río Gallegos Apart & Suites
chonchi - nearest hotel: Hotel Huildín
essaouira - nearest hotel: Essaouira Wind Palace
buckeye - nearest hotel: Days Inn
sarmiento - nearest hotel: Ismar
kill devil hills - near

,City,Country,Lat,Lng,Humidity,Hotel Name
11,labrador city,CA,52.9463,-66.9114,51,Carol Inn
17,an nuhud,SD,12.7000,28.4333,11,No hotel found
26,kodiak,US,57.7900,-152.4072,48,No hotel found
34,chibougamau,CA,49.9168,-74.3659,65,No hotel found
68,tomatlan,MX,19.9333,-105.2500,89,Hotel Hacienda Vieja Tomatlan
...,...,...,...,...,...,...
540,raton,US,36.9034,-104.4391,36,Travelodge
543,cuatro cienegas de carranza,MX,26.9833,-102.0833,15,No hotel found
552,la cruz,MX,23.9167,-106.9000,71,Hotel Las Palmas
562,crane,US,31.3974,-102.3501,27,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# Filter hotel_df to exclude rows where "Hotel Name" is "No hotel found"
valid_hotels_df = hotel_df[hotel_df['Hotel Name'] != 'No hotel found']

# Plot the valid hotels using hvplot
plot = valid_hotels_df.hvplot.points(x='Lng', y='Lat', geo=True, color='City', size=100, alpha=0.6, tiles = 'OSM', hover_cols=['City', 'Hotel Name'])

# Show the plot
plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)

In [9]:
valid_hotels_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,labrador city,CA,52.9463,-66.9114,51,Carol Inn
68,tomatlan,MX,19.9333,-105.2500,89,Hotel Hacienda Vieja Tomatlan
72,nerac,FR,44.1333,0.3500,87,Les Terrasses du Petit Nérac
81,saint-francois,FR,46.4154,3.9054,100,Chez Lily
83,conde,BR,-7.2597,-34.9075,94,Paladino
97,saint-pierre,RE,-21.3393,55.4781,65,Tropic Hotel
117,mountain home,US,43.1330,-115.6912,56,Best Western Foothills Inn
126,port elizabeth,ZA,-33.9180,25.5701,85,Waterford Hotel
130,broken hill,AU,-31.9500,141.4333,42,Ibis Styles
152,walvis bay,NaN,-22.9575,14.5053,100,Atlantic
